# One Way Normal Model in PyMC3

In [16]:
from timeit import default_timer as timer
import numpy as np
import pickle

import pymc3 as pm

from utils import generate_datasets, SEED, I, SIGMA

## 1. Models

In [15]:
# Centered 
def pymc3_model_c(y, I=I, sigma=SIGMA):
  model = pm.Model()

  with model:
    mu = pm.Normal('mu', mu=0, sd=5);
    tau = pm.HalfCauchy('tau', beta=2.5);
    theta = pm.Normal('theta', mu=mu, sd=tau, shape=I);
    y_obs = pm.Normal('y_obs', mu=theta, sd=sigma, observed=y);
    
  return model
  
# Non-centered
def pymc3_model_nc(y, I=I, sigma=SIGMA):
  model = pm.Model()

  with model:
    mu = pm.Normal('mu', mu=0, sd=5);
    tau = pm.HalfCauchy('tau', beta=2.5);
    vtheta = pm.Normal('vtheta', mu=0, sd=1, shape=I);
    y_obs = pm.Normal('y_obs', mu=tau*vtheta + mu, sd=sigma, observed=y);
    
  return model

## 2. Inference

### NUTS

In [ ]:
def pymc3_nuts_c(seeds=SEED):
  """
  Runs PyMC3's NUTS algorithm for centered parameterization
  Default parameters: configuration from the paper
  """
  Y, theta = generate_datasets(seeds=seeds)
  
  for y,seed in zip(Y, seeds):
    print(seed)
    pm_model_c = pymc3_model_c(I, sigma, y, seed)
    
    with pm_model_c:
      start = timer()
      # In stan: total iters 100000 (incl.warmup) 
      trace = pm.sample(draws=95000, tune=5000, chains=4, random_seed=seed, nuts_kwargs=dict(target_accept=0.999))
      end = timer()
      trace_thin = trace[::1000]
      mu = np.array(trace_thin.get_values('mu', combine=False)).mean(axis=0)
      tau = np.array(trace_thin.get_values('tau', combine=False)).mean(axis=0)
      results = {'mu': mu, 'tau': tau, 'time': end-start, 'iters': 100000, 'tune': 5000, 
                 'thin': 1000, 'divergences': int(trace_thin['diverging'].nonzero()[0].size)}
      with open('results/pymc3/nuts_c_{}.pkl'.format(seed), 'wb') as f:
        pickle.dump(results, f)
  print('Done')

In [ ]:
def pymc3_nuts_nc(iters=45000, tune=5000, adapt_delta=0.8, mode='nominal', seeds=SEED):
  """
  Runs PyMC3's NUTS algorithm for centered parameterization
  Default parameters: nominal configuration from the paper
  For baseline use: iters=95000, tune=5000, adapt_delta=0.99, mode='baseline'
  """
  Y, theta = generate_datasets(seeds=seeds)
  
  for y,seed in zip(Y, seeds):
    print(seed)
    pm_model_c = pymc3_model_c(I, sigma, y, seed)
    
    with pm_model_c:
      start = timer()
      # In stan: total iters 50000 (incl.warmup) for nominal, 100000 for baseline 
      trace = pm.sample(draws=iters, tune=tune, chains=4, random_seed=seed, nuts_kwargs=dict(target_accept=adapt_delta))
      end = timer()
      results = {'mu': trace['mu'], 'tau': trace['tau'], 'time': end-start, 'iters': iters, 
                 'tune': tune, 'divergences': int(trace['diverging'].nonzero()[0].size)}
      with open('results/pymc3/nuts_nc_{}_{}.pkl'.format(mode, seed), 'wb') as f:
        pickle.dump(results, f)
  print('Done')

In [ ]:
# centered
pymc3_nuts_c()

In [ ]:
# nominal
pymc3_nuts_nc()

In [ ]:
# baseline
pymc3_nuts_nc(iters=95000, tune=5000, adapt_delta=0.99, mode='baseline')

## ADVI

In [33]:
def pymc3_vi(mode='c', seeds=SEED):
  """
  Runs PyMC3 ADVI algorithm
  If mode is 'c', use the centered parameterization
  If mode is 'nc', use the non-centered parameterization
  """
  if mode not in ['c', 'nc']:
    raise "Mode has to be 'c' for centered or 'nc'"
  Y, theta = generate_datasets(seeds=seeds)
  for seed, y in zip(seeds, Y):
    model = pymc3_model_c(y) if mode == 'c' else pymc3_model_nc(y) 
    iters = np.linspace(50000, 500000, 5).astype(int)
    for n in iters:
      with model:
        print('...')
        start = timer()
        advi_fit = pm.fit(n=n, random_seed=seed, callbacks=[pm.callbacks.CheckParametersConvergence(diff='absolute', tolerance=0.0001)])
        end = timer()
        trace = advi_fit.sample(draws=1000)
        mu_ = np.array(trace.get_values('mu'))
        tau_ = np.array(trace.get_values('tau'))
                      
        results = {'iters': n, 'tol': 0.0001, 'time': end-start, 'mu': mu_, 'tau': tau_}
        pickle.dump(results, open('results/pymc3/vi_{}_{}.pkl'.format(mode, seed), 'ab'))
    print('Done')

In [ ]:
pymc3_vi(mode='c')

In [ ]:
pymc3_vi(mode='nc')

### References

[1] Betancourt, Michael J. and Girolami, Mark. Hamiltonian Monte Carlo for Hierarchical Models. 2013.